# Dashboard de Segmentación de Clientes con Insights Visuales Exclusivos y Predicción

- Cada cluster tiene su card visual, insights con iconos y colores.
- Solo los clusters top de cada métrica reciben el insight, sin repeticiones.
- Predicción de cluster, exportación, KPIs y visualizaciones.
- Ejecuta en JupyterLab/VSCode (no Jupyter clásico), o como script `.py`.

In [2]:
import os
import webbrowser
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA, NMF
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import plotly.express as px
import plotly.graph_objects as go
from dash import Dash, dcc, html, Input, Output, State, dash_table, no_update
import dash_bootstrap_components as dbc
from scipy.stats import entropy
import datetime as dt
import warnings
warnings.filterwarnings('ignore')

## Carga y preprocesamiento de datos

In [3]:
file_path = "Data_Set_Global.xlsx"
assert os.path.exists(file_path), f"No se encuentra el archivo {file_path}"
pedidos = pd.read_excel(file_path, sheet_name="Pedidos")
pedidos["total_price"] = pedidos["quantity"] * pedidos["unit_price"]
pedidos["order_date"] = pd.to_datetime(pedidos["order_date"])
for col in ["promised_delivery_time", "actual_delivery_time"]:
    if col in pedidos.columns:
        pedidos[col] = pd.to_datetime(pedidos[col])
if "promised_delivery_time" in pedidos.columns and "actual_delivery_time" in pedidos.columns:
    pedidos["delivery_delay_min"] = (
        (pedidos["actual_delivery_time"] - pedidos["promised_delivery_time"]).dt.total_seconds() / 60
    )
pedidos = pedidos.dropna(subset=["order_id", "customer_id", "order_date"]).copy()
pedidos.reset_index(drop=True, inplace=True)

## Funciones de clustering y generación de insights visuales

In [4]:
def preparar_segmentacion(pedidos_filtrado, K=3, nmf_topics=6):
    snapshot_date = pedidos_filtrado["order_date"].max() + dt.timedelta(days=1)
    rfm = pedidos_filtrado.groupby("customer_id").agg(
        recency=("order_date", lambda date: (snapshot_date - date.max()).days),
        frequency=("order_id", "nunique"),
        monetary=("total_price", "sum")
    )
    scaler_rfm = StandardScaler()
    rfm_z = pd.DataFrame(scaler_rfm.fit_transform(rfm), index=rfm.index, columns=[c+"_z" for c in rfm.columns])
    if "payment_method" in pedidos_filtrado.columns:
        pay_counts = pedidos_filtrado.groupby(["customer_id", "payment_method"]).size().unstack(fill_value=0)
        pay_probs = pay_counts.div(pay_counts.sum(axis=1), axis=0)
        payment_entropy = pay_probs.apply(lambda row: entropy(row, base=2), axis=1)
        customer_payment_diversity = payment_entropy.rename("payment_entropy").reset_index()
    else:
        customer_payment_diversity = pd.DataFrame({"customer_id": rfm.index, "payment_entropy": 0})
    cust_prod = pedidos_filtrado.groupby(["customer_id", "product_id"]).size().unstack(fill_value=0)
    nmf = NMF(n_components=min(nmf_topics, len(cust_prod.columns)), init="random", random_state=42)
    W = nmf.fit_transform(cust_prod)
    customer_topics = pd.DataFrame(W, index=cust_prod.index, columns=[f"topic_{i+1}" for i in range(W.shape[1])])
    full_df = (
        rfm_z
        .join(customer_payment_diversity.set_index("customer_id"), how="left")
        .join(customer_topics, how="left")
    ).fillna(0)
    X_scaled = StandardScaler().fit_transform(full_df.select_dtypes(include=np.number))
    kmeans = KMeans(n_clusters=K, random_state=42, n_init=10)
    cluster_labels = kmeans.fit_predict(X_scaled)
    full_df["cluster"] = cluster_labels
    full_df.index.name = "customer_id"
    profile_cols = [c for c in full_df.columns if c.startswith(('recency','frequency','monetary','payment_entropy','topic_'))]
    cluster_profile = full_df.groupby("cluster")[profile_cols].mean().round(2)
    pca = PCA(n_components=2)
    coords = pca.fit_transform(X_scaled)
    df_plot = full_df.reset_index().copy()
    df_plot['PCA1'] = coords[:,0]
    df_plot['PCA2'] = coords[:,1]
    return full_df, cluster_profile, kmeans, scaler_rfm, customer_topics, df_plot, X_scaled

def get_cluster_names(K):
    nombres = {
        0: "Champions 🏆",
        1: "Dormant 💤",
        2: "Ocasionales 🎯",
        3: "Leales 💳",
        4: "Prometedores 💡"
    }
    return {i: nombres.get(i, f"Cluster {i}") for i in range(K)}

def get_automatic_insights(cluster_profile, dataset_profile, cluster_names):
    top_payment = cluster_profile['payment_entropy'].idxmax()
    top_monetary = cluster_profile['monetary_z'].idxmax()
    top_freq = cluster_profile['frequency_z'].idxmax()
    min_recency = cluster_profile['recency_z'].idxmin()
    icon_style = {
        "payment": {"icon": "💳", "color": "#1976D2"},
        "monetary": {"icon": "💰", "color": "#388E3C"},
        "freq": {"icon": "🔁", "color": "#F9A825"},
        "recency": {"icon": "⏰", "color": "#C2185B"},
        "neutral": {"icon": "🔎", "color": "#757575"}
    }
    cards = []
    for cid, row in cluster_profile.iterrows():
        insights = []
        name = cluster_names.get(cid, f"Cluster {cid}")
        if cid == top_payment:
            insights.append(html.Li([
                html.Span(icon_style["payment"]["icon"], style={"color": icon_style["payment"]["color"], "marginRight": "7px"}),
                html.Span("Mayor diversidad de métodos de pago: es el más multicanal.")
            ]))
        if cid == top_monetary:
            insights.append(html.Li([
                html.Span(icon_style["monetary"]["icon"], style={"color": icon_style["monetary"]["color"], "marginRight": "7px"}),
                html.Span("Mayor gasto total: prioriza ofertas premium y lealtad.")
            ]))
        if cid == top_freq:
            insights.append(html.Li([
                html.Span(icon_style["freq"]["icon"], style={"color": icon_style["freq"]["color"], "marginRight": "7px"}),
                html.Span("Mayor frecuencia de compra: ideal para programas de suscripción o retención.")
            ]))
        if cid == min_recency:
            insights.append(html.Li([
                html.Span(icon_style["recency"]["icon"], style={"color": icon_style["recency"]["color"], "marginRight": "7px"}),
                html.Span("Han comprado más recientemente: oportunidad inmediata de cross-selling.")
            ]))
        if not insights:
            insights.append(html.Li([
                html.Span(icon_style["neutral"]["icon"], style={"color": icon_style["neutral"]["color"], "marginRight": "7px"}),
                html.Span("Comportamiento similar al promedio del dataset.")
            ]))
        cards.append(
            html.Div([
                html.H5([
                    html.Span(name, style={"color": "#1976D2"}),
                    f"  (Cluster {cid})"
                ], style={"marginBottom":"0.3em"}),
                html.Ul(insights, style={"marginBottom": "0.3em"})
            ], style={
                "background": "#fafdff", "borderRadius": "10px", "padding": "0.8em 1.2em",
                "margin": "0.5em 0", "boxShadow": "0 2px 8px #eaf6fb", "border": "1.5px solid #eaf6fb"
            })
        )
    return html.Div([
        html.H4([
            html.Span("🧠 Insights automáticos por cluster", style={"color":"#6b49b8"})
        ]),
        html.P("Solo el cluster top en cada métrica recibe el insight destacado. Así evitas repeticiones.", style={"fontSize": "1em", "color": "#6b7b8c"}),
        html.Div(cards)
    ], style={"background": "#eaf6fb", "borderRadius": "10px", "padding": "1.3em", "marginBottom": "2em"})

def elbow_silhouette(X):
    inertias, silhouettes, Ks = [], [], range(2,11)
    for k in Ks:
        km = KMeans(n_clusters=k, random_state=42, n_init=10).fit(X)
        inertias.append(km.inertia_)
        silhouettes.append(silhouette_score(X, km.labels_))
    fig1 = go.Figure([go.Scatter(x=list(Ks), y=inertias, mode='lines+markers')])
    fig1.update_layout(title="Método del codo — ¿Cuántos clusters elegir?", xaxis_title="K", yaxis_title="Inercia",
                      annotations=[dict(text="Busca el 'codo' para el K ideal", x=6, y=inertias[4], showarrow=True, arrowhead=1)])
    fig2 = go.Figure([go.Scatter(x=list(Ks), y=silhouettes, mode='lines+markers')])
    fig2.update_layout(title="Silhouette promedio — Calidad del agrupamiento", xaxis_title="K", yaxis_title="Coef. silhouette",
                      annotations=[dict(text="Mayor silhouette = clusters mejor definidos", x=4, y=max(silhouettes), showarrow=True, arrowhead=1)])
    return fig1, fig2

## Layout del Dashboard

In [5]:
app = Dash(__name__, external_stylesheets=[dbc.themes.MINTY])
min_date = pedidos['order_date'].min().date()
max_date = pedidos['order_date'].max().date()

recency_opts = [("0-7 días", 3), ("8-30 días", 15), ("31-90 días", 45), ("+90 días", 120)]
freq_opts = [("1-3 compras", 2), ("4-6 compras", 5), ("7+ compras", 8)]
monetary_opts = [("$1-300", 150), ("$301-700", 500), ("$701+ ", 900)]
payment_entropy_opts = [("1 (pago único)", 0.0), ("2", 0.5), ("3 o más", 0.95)]

app.layout = dbc.Container([
    html.H2("🧑‍💻 Dashboard Segmentación Clientes — Insights Exclusivos & Predicción"),
    html.P("Segmenta clientes y predice el cluster de un nuevo perfil con solo 1 clic."),
    dbc.Row([
        dbc.Col([
            html.Label('Rango de fechas:'),
            dcc.DatePickerRange(
                id='date-range',
                min_date_allowed=min_date,
                max_date_allowed=max_date,
                start_date=min_date,
                end_date=max_date
            ),
            html.Div("Todo el análisis se recalcula al cambiar la fecha.", style={"fontSize":"0.92em", "color":"#888"})
        ], width=6),
        dbc.Col([
            html.Label('Número de clusters (K):'),
            dcc.Slider(id='slider-k', min=2, max=5, step=1, value=3,
                       marks={i:str(i) for i in range(2,6)}, tooltip={"placement":"bottom"})
        ], width=6)
    ]),
    html.Br(),
    dcc.Loading(html.Div(id='kpi-panel')),
    html.Br(),
    html.Div(id='auto-insights'),
    html.Hr(),
    dbc.Row([
        dbc.Col([dcc.Loading(dcc.Graph(id='fig-pca'))], width=6),
        dbc.Col([dcc.Loading(dcc.Graph(id='fig-bar-cluster'))], width=6),
    ]),
    html.Br(),
    html.H3("Predicción de segmento para nuevo cliente"),
    html.P("Completa el perfil para predecir a qué cluster pertenecería un nuevo cliente. Consulta la explicación y sugerencia de acción."),
    dbc.Row([
        dbc.Col([
            dcc.Dropdown(options=[{"label":t,"value":v} for t,v in recency_opts], id="input-recency", placeholder="Recency (días)")
        ], width=3),
        dbc.Col([
            dcc.Dropdown(options=[{"label":t,"value":v} for t,v in freq_opts], id="input-frequency", placeholder="Frecuencia")
        ], width=3),
        dbc.Col([
            dcc.Dropdown(options=[{"label":t,"value":v} for t,v in monetary_opts], id="input-monetary", placeholder="Monetary")
        ], width=3),
        dbc.Col([
            dcc.Dropdown(options=[{"label":t,"value":v} for t,v in payment_entropy_opts], id="input-payment-entropy", placeholder="Diversidad pago")
        ], width=3),
    ]),
    html.Br(),
    html.Button("Predecir cluster", id="boton-predecir", n_clicks=0, style={"marginBottom":10, "backgroundColor":"#90d6b0", "border":"none", "padding":"8px 20px", "borderRadius":"6px"}),
    html.Div(id="prediccion-resultado", style={"fontSize":"1.1em", "marginTop":"1em"}),
    html.Hr(),
    html.H4("Tabla de clientes (con cluster asignado)"),
    dash_table.DataTable(
        id='clientes-table',
        page_size=10, filter_action="native", sort_action="native", style_table={'overflowX': 'auto'}, style_cell={'textAlign': 'center'},
        style_header={'backgroundColor': '#f8f9fa', 'fontWeight': 'bold'},
        style_data_conditional=[
            {
                'if': {'row_index': 'odd'},
                'backgroundColor': '#f3f8fb'
            }
        ]
    ),
    html.Button("Exportar tabla a CSV", id="export-btn", n_clicks=0, style={"marginLeft":10}),
    dcc.Download(id="download-clients"),
    html.Br(),
    html.Footer([
        html.Span("© 2025 | Dashboard por AlejandroRS1 + Copilot AI")
    ], style={"textAlign":"center", "color":"#888"})
], fluid=True)

## Callbacks principales

In [6]:
@app.callback(
    [Output('kpi-panel','children'),
     Output('auto-insights','children'),
     Output('fig-pca','figure'),
     Output('fig-bar-cluster','figure'),
     Output('fig-codo','figure'),
     Output('fig-silhouette','figure'),
     Output('clientes-table','columns'),
     Output('clientes-table','data'),
     Output('boton-predecir','disabled')],
    [Input('date-range','start_date'),
     Input('date-range','end_date'),
     Input('slider-k','value')]
)
def update_dashboard(start_date, end_date, K):
    global scaler_rfm_global, kmeans_global
    pedidos_filt = pedidos[(pedidos['order_date']>=pd.to_datetime(start_date)) & (pedidos['order_date']<=pd.to_datetime(end_date))].copy()
    kpi_dict = {
        "Clientes Analizados": pedidos_filt["customer_id"].nunique(),
        "Órdenes": pedidos_filt['order_id'].nunique(),
        "Facturación ($)": round(pedidos_filt['total_price'].sum(),2),
        "Periodo": f"{str(start_date)} a {str(end_date)}",
        "K actual": K
    }
    kpi_panel = dbc.Row([
        dbc.Col(
            dbc.Card([
                dbc.CardHeader(k),
                dbc.CardBody(html.H5(str(v)))
            ]), width=2
        ) for k, v in kpi_dict.items()
    ])
    full_df, cluster_profile, kmeans, scaler_rfm, customer_topics, df_plot, X_scaled = preparar_segmentacion(pedidos_filt, K)
    cluster_names = get_cluster_names(K)
    dataset_profile = full_df[["payment_entropy", "monetary_z", "frequency_z", "recency_z"]].mean()
    auto_insights_panel = get_automatic_insights(cluster_profile, dataset_profile, cluster_names)
    fig_pca = px.scatter(df_plot, x='PCA1', y='PCA2', color=df_plot['cluster'].astype(str), symbol=df_plot['cluster'].astype(str),
                        title="Mapa visual de clusters de clientes",
                        labels={"color": "Cluster"}, hover_data=['customer_id'])
    fig_pca.update_traces(marker=dict(line=dict(width=2, color='DarkSlateGrey')))
    cluster_counts = full_df['cluster'].value_counts().sort_index()
    fig_bar = px.bar(x=cluster_counts.index, y=cluster_counts.values,
                     labels={'x':'Cluster','y':'Cantidad de clientes'}, title='Clientes por Cluster',
                     color=cluster_counts.index.astype(str), color_discrete_sequence=px.colors.qualitative.Vivid)
    fig_codo, fig_silhouette = elbow_silhouette(X_scaled)
    columns = [{"name": i.replace('_z',' (Z)').replace('_',' ').title(), "id": i} for i in ["recency_z","frequency_z","monetary_z","payment_entropy"]+list(customer_topics.columns)+["cluster"]]
    data = full_df.reset_index()[["customer_id","recency_z","frequency_z","monetary_z","payment_entropy"]+list(customer_topics.columns)+["cluster"]].to_dict('records')
    scaler_rfm_global = scaler_rfm
    kmeans_global = kmeans
    return kpi_panel, auto_insights_panel, fig_pca, fig_bar, fig_codo, fig_silhouette, columns, data, False

In [7]:
@app.callback(
    Output('prediccion-resultado', 'children'),
    Input('boton-predecir', 'n_clicks'),
    State('input-recency', 'value'),
    State('input-frequency', 'value'),
    State('input-monetary', 'value'),
    State('input-payment-entropy', 'value'),
    prevent_initial_call=True
)
def predecir_cluster(n_clicks, recency, frequency, monetary, payment_entropy):
    if n_clicks:
        if None in [recency, frequency, monetary, payment_entropy]:
            return html.Span("Completa todos los campos para predecir el cluster.", style={"color":"#c33"})
        X_new = np.array([[recency, frequency, monetary]])
        X_new_scaled = scaler_rfm_global.transform(X_new)
        X_pred = np.concatenate([X_new_scaled, np.array([[payment_entropy]])], axis=1)
        cluster = kmeans_global.predict(X_pred)[0]
        return html.Span([
            f"El nuevo cliente pertenecería al cluster {cluster}. ",
            html.B(f"{get_cluster_names(kmeans_global.n_clusters)[cluster]}")
        ], style={"color":"#237a3a"})
    return ""

In [ ]:
@app.callback(
    Output('download-clients', 'data'),
    Input('export-btn', 'n_clicks'),
    State('clientes-table', 'data'),
    prevent_initial_call=True
)
def export_csv(n_clicks, table_data):
    if n_clicks:
        df_export = pd.DataFrame(table_data)
        if not df_export.empty:
            return dcc.send_data_frame(df_export.to_csv, "clientes_segmentados.csv", index=False)
    return no_update

In [8]:
if __name__ == "__main__":
    import threading, time
    def abrir_browser():
        time.sleep(1)
        webbrowser.open("http://127.0.0.1:8052/")
    threading.Thread(target=abrir_browser).start()
    print("Abriendo tu dashboard en http://127.0.0.1:8052/ ...")
    app.run_server(debug=True, port=8052)

Abriendo tu dashboard en http://127.0.0.1:8052/ ...


ObsoleteAttributeException: app.run_server has been replaced by app.run